In [22]:
import requests
import pandas as pd

In [23]:
# Send a GET request to the CoinGecko API
response = requests.get("https://api.coingecko.com/api/v3/simple/supported_vs_currencies")

In [24]:
response.status_code

200

In [25]:
# Retrieve the JSON data from the response
vs_currencies = response.json()

In [26]:
# Currencies used to buy crypto (includes fiat (JPY, USD)), for some reason excludes stablecoins
print(vs_currencies)

['btc', 'eth', 'ltc', 'bch', 'bnb', 'eos', 'xrp', 'xlm', 'link', 'dot', 'yfi', 'usd', 'aed', 'ars', 'aud', 'bdt', 'bhd', 'bmd', 'brl', 'cad', 'chf', 'clp', 'cny', 'czk', 'dkk', 'eur', 'gbp', 'hkd', 'huf', 'idr', 'ils', 'inr', 'jpy', 'krw', 'kwd', 'lkr', 'mmk', 'mxn', 'myr', 'ngn', 'nok', 'nzd', 'php', 'pkr', 'pln', 'rub', 'sar', 'sek', 'sgd', 'thb', 'try', 'twd', 'uah', 'vef', 'vnd', 'zar', 'xdr', 'xag', 'xau', 'bits', 'sats']


In [27]:
r = requests.get('https://api.coingecko.com/api/v3/coins/list')
coins = pd.DataFrame(r.json())

In [28]:
coins

,id,symbol,name
0,01coin,zoc,01coin
1,0chain,zcn,Zus
2,0vix-protocol,vix,0VIX Protocol
3,0x,zrx,0x Protocol
4,0x0-ai-ai-smart-contract,0x0,0x0.ai: AI Smart Contract
...,...,...,...
10009,zynergy,zyn,Zynergy
10010,zyro,zyro,Zyro
10011,zyrri,zyr,Zyrri
10012,zyx,zyx,ZYX


In [41]:
def get_VE(coins, vs_currencies, base='BTC',target='USDT'):
    
    global test_X, test_r
    
    if target.lower() not in vs_currencies:
        print('Warning target currency not in vs_currencies:{}'.format(target))

    
    try:
        coinid = list(coins[coins['symbol']==base.lower()]['id'])[0]
        print(coinid)
    except:
        print('Invalid base coin:{}'.format(base))
        return None, -2

    api_call = 'https://api.coingecko.com/api/v3/coins/{}/tickers'.format(coinid)
    try:
        print(api_call)
        r = requests.get(api_call)
    except:
        print('API endpoint unreachable'.format(r.status_code))
        return None, -3
    
    if r.status_code==429:
        print('Too many requests')
        return None, 429
    if r.status_code==200:
        try:
            X = pd.DataFrame(r.json()['tickers'])
        except:
            print('Cannot convert json payload to dataframe')
            # dump r to json
            return None, -4

        if X.empty:
            print('No such pair exists: {}-{}'.format(base, target))
            return None, 0
        
        X = X[X['target']==target]
        
        if X.empty:
            print('No such pair exists: {}-{}'.format(base, target))
            return None, 0

        try:
            X['exchange_name'] = X['market'].transform(lambda x: x['name'])
        except:
            print('Cannot extract exchange name')
            X['exchange_name'] = None
        return X, 200
   
    

In [60]:
Z = get_VE(coins,vs_currencies, base='ETH',target='BTC')
print('Status code:', Z[1])

ethereum
https://api.coingecko.com/api/v3/coins/ethereum/tickers
Status code: 200


In [61]:
Z[0].head()

,base,target,market,last,volume,converted_last,converted_volume,trust_score,bid_ask_spread_percentage,timestamp,last_traded_at,last_fetch_at,is_anomaly,is_stale,trade_url,token_info_url,coin_id,target_coin_id,exchange_name
8,ETH,BTC,"{'name': 'Binance', 'identifier': 'binance', '...",0.069560,14258.982141,"{'btc': 0.06956, 'eth': 1.000155, 'usd': 1843.17}","{'btc': 991.855, 'eth': 14261, 'usd': 26281715}",green,0.014376,2023-06-09T18:30:36+00:00,2023-06-09T18:30:36+00:00,2023-06-09T18:30:36+00:00,False,False,https://www.binance.com/en/trade/ETH_BTC?ref=3...,None,ethereum,bitcoin,Binance
12,ETH,BTC,"{'name': 'Pionex', 'identifier': 'pionex', 'ha...",0.069536,8246.439100,"{'btc': 0.069536, 'eth': 0.99950036, 'usd': 18...","{'btc': 572.393, 'eth': 8227, 'usd': 15160603}",green,0.017190,2023-06-09T18:34:24+00:00,2023-06-09T18:34:24+00:00,2023-06-09T18:34:24+00:00,False,False,https://www.pionex.com/en/trade/ETH_BTC/Bot,None,ethereum,bitcoin,Pionex
14,ETH,BTC,"{'name': 'Tidex', 'identifier': 'tidex', 'has_...",0.069573,1262.309758,"{'btc': 0.06957348, 'eth': 0.99990459, 'usd': ...","{'btc': 87.823, 'eth': 1262, 'usd': 2323063}",green,0.010004,2023-06-09T18:35:23+00:00,2023-06-09T18:35:23+00:00,2023-06-09T18:35:23+00:00,False,False,None,None,ethereum,bitcoin,Tidex
18,ETH,BTC,"{'name': 'Bitstamp', 'identifier': 'bitstamp',...",0.069554,402.299884,"{'btc': 0.06955352, 'eth': 1.000034, 'usd': 18...","{'btc': 27.981373, 'eth': 402.314, 'usd': 741283}",green,0.040661,2023-06-09T18:32:59+00:00,2023-06-09T18:32:59+00:00,2023-06-09T18:32:59+00:00,False,False,https://www.bitstamp.net/markets/eth/btc/,None,ethereum,bitcoin,Bitstamp
30,ETH,BTC,"{'name': 'Ace', 'identifier': 'ace', 'has_trad...",0.069575,125.296000,"{'btc': 0.069575, 'eth': 1.000061, 'usd': 1842...","{'btc': 8.700116, 'eth': 125.054, 'usd': 230434}",green,0.061798,2023-06-09T18:34:17+00:00,2023-06-09T18:34:17+00:00,2023-06-09T18:34:17+00:00,False,False,https://ace.io/trade/ETH/BTC,None,ethereum,bitcoin,Ace


In [64]:
print('Some of the exchanges where you can buy ETH with BTC')
list(Z[0].head(10)['exchange_name'])

Some of the exchanges where you can buy ETH with BTC


['Binance',
 'Pionex',
 'Tidex',
 'Bitstamp',
 'Ace',
 'Coinsbit',
 'XT.COM',
 'Coinbase Exchange',
 'Txbit',
 'OKX']

In [58]:
Z = get_VE(coins,vs_currencies, base='ETH',target='PLN')
print('Status code:', Z[1])

ethereum
https://api.coingecko.com/api/v3/coins/ethereum/tickers
No such pair exists: ETH-PLN
Status code: 0


Cannot buy ethereum directly with Zloty